1. One model that emits a random number between 0 and 25 every ten seconds.
2. One model that emits a line in some file (say, from project gutenberg or something, it doesn't matter) one at a time.
3. One model that receives a number and a line of text, and then it applies something to them and emits a new line of text.  Have it rotate the text -- if you're familiar with ROT13, this would be ROTXX where XX is the random number.

Each of these can be a python function, and I want you to compose a graph of them.

We will run these inside yggdrasil, and each one will have a widget that will display its output.

In [2]:
import jupyterlab_nodeeditor as jlne
from yggdrasil.interface.YggInterface import YggInput, YggOutput
from yggdrasil.runner import YggRunner
import numpy as np
import time

In [5]:
def model_rng():
    # Setup output file
    out_file = YggOutput('outputRNG_file')

    # This variable is jsut to limit the number of numbers output, can remove if needed
    fake_timer = 0

    # Every 10 seconds, send a random number through the model
    while True:
        if fake_timer == 3:
            break
        rng = np.random.randint(0, 25)
        out_file.send(rng)
        fake_timer += 1
        time.sleep(10)

In [9]:
# If we want to do remote reading, we can try reading in the url as its own model
# Then transfer that model to another model to read it in ygg
# This is done locally

def model_reader():
    in_file = YggInput('inputReader_file')
    out_file = YggOutput('outputReader_file')
    line_number = 0
    ret = True

    # Output the text line by line with the line number
    while ret:
        (ret, line) = in_file.recv()
        if ret:
            line_number += 1
            ret = out_file.send(line_number, line)
            if not ret:
                raise RuntimeError("ERROR SENDING LINE")
        else:
            print("End of File")
            out_file.send_eof()

In [14]:
# Input a number for rotation and a string via file
def rotxx_func(x, txt):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    txt_string = txt.decode("utf-8")

    # It's a one-liner: rotate the string by the number and send it out
    rotxx = "".join([alphabet[(alphabet.find(c) + x) % 26] for c in txt_string])
    return rotxx